# Step1. Import all required libraries

In [ ]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
# os.chdir('C:\Users\Rajat Rai\Projects\LogosCNN\Image Scrapper')

# Step2. Install Chrome Driver

In [ ]:
#Install driver
opts=webdriver.ChromeOptions()
opts.headless=True

driver = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)


# Step4.  Write a function to take cursor to end of the page

In [ ]:
def scroll_to_end(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)#sleep_between_interactions   
        

# Step5. Write a function to get url of each Image

In [ ]:
#no license issues

def getImageUrls(name,totalImgs,driver):
    
    search_url = "https://www.google.com/search?q={q}&tbm=isch&ved=2ahUKEwj126a6gM_sAhWGSKwKHSPxDz4Q2-cCegQIABAA&oq={q}&gs_lcp=CgNpbWcQAzICCAAyAggAMgIIADIGCAAQBRAeMgYIABAIEB4yBggAEAoQGDoHCCMQ6gIQJzoECCMQJzoFCAAQsQM6CAgAELEDEIMBOgQIABBDOgQIABAYUOnsCliPhAtgpIYLaAFwAHgAgAGRAYgB2weSAQM5LjKYAQCgAQGqAQtnd3Mtd2l6LWltZ7ABCsABAQ&sclient=img&ei=DwyVX7WYJ4aRsQWj4r_wAw&bih=734&biw=1579&hl=en"
    driver.get(search_url.format(q=name))

    img_urls = set()
    img_count = 0
    results_start = 0  
    
    while(img_count<totalImgs): #Extract actual images now
        
        scroll_to_end(driver)
        
        thumbnail_results = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
        totalResults=len(thumbnail_results)
        print(f"Found: {totalResults} search results. Extracting links from {results_start}:{totalResults}")
        
        for img in thumbnail_results[results_start:totalResults]:
            try:
                img.click()
                time.sleep(2)
                actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
                for actual_image in actual_images:
                    if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                        img_urls.add(actual_image.get_attribute('src'))
                
                img_count=len(img_urls)
                
                if img_count >= totalImgs:
                    print(f"Found: {img_count} image links")
                    break
                else:
                    print("Found:", img_count, "looking for more image links ...")                
                    load_more_button = driver.find_element_by_css_selector(".mye4qd")
                    driver.execute_script("document.querySelector('.mye4qd').click();")
                    results_start = len(thumbnail_results)
            except:
                pass
    return img_urls
        

# Step6. Write a function to download each Image

In [ ]:

def downloadImages(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = folder_path + "\\" + file_name
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

# Step7. Write a function to save each Image in Destination directory

In [ ]:

def saveInDestFolder(searchNames,destDir,totalImgs,driver):
    for name in searchNames:
        # path=os.path.join(destDir,name)
        path=  destDir+name
        # print('Current Path',path)
        if not os.path.exists(path):
            os.mkdir(path)
        print('Current Path',path)
        totalLinks=getImageUrls(name+" logo",totalImgs,driver)
        print('totalLinks',totalLinks)

        if totalLinks is None:
            print('images not found for :',name)
            continue
        else:
            for i, link in enumerate(totalLinks):
                file_name = str(i)+".jpg"
                downloadImages(path,file_name,link)
            
searchNames=['Altera', 'Analog Devices', 'Digilent Inc', 'Fairchild Semiconductor', 'FTDI', 'General Semiconductors Industries Inc', 'Harris Semiconductors', 'Infineon Technologies', 'Integrated Device Technology', 'Level One', 'Linear Technologies', 'Maxim Integrated Circuit', 'Micron Technologies Inc', 'Mitsubishi Electric Corporation', 'Nvidia', 'ON', 'Pericom Semiconductors', 'Pulse-a Technitrol Co', 'Realtek Semiconductors', 'Samsung Electronics', 'SGS-Thomson', 'Spansion', 'Summit Microelectronics', 'Texas Instruments Inc', 'Xilinx', 'Zilker labs']
destDir='.\\GoogleImages\\'
totalImgs=5

saveInDestFolder(searchNames,destDir,totalImgs,driver)





In [ ]:
##TODO:
## Image formats : svg/transparent -> download correctly
## Keyword combinations For example : Logo, Secicomductor logos, IC 
## 